# CasualMarket MCP 工具測試手冊



In [6]:
from agents.mcp import MCPServerStdio
import json

# 修復後的參數設定 - 這是正確的配置
params = {
    "command": "uv",
    "args": [
        "run",
        "--directory", "/Users/sacahan/Documents/workspace/CasualMarket",
        "python", "-m", "src.main"
    ],
    "env": {
        "MARKET_MCP_LOG_FILE": "./logs/mcp.log"
    }
}

## 1. 交易相關工具測試

### 1.1 取得台灣股票即時價格

In [5]:
# 測試不同的股票代碼（MCP 連接已修復）- 美化 JSON 結果顯示
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "2317"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))


{
  "success": true,
  "data": {
    "symbol": "2317",
    "company_name": "鴻海精密工業股份有限公司",
    "current_price": 239.0,
    "change": 0.5,
    "change_percent": 0.0020964360587002098,
    "volume": 117673,
    "high": 245.5,
    "low": 238.0,
    "open": 241.5,
    "previous_close": 238.5,
    "last_update": "2025-10-21T14:30:00"
  },
  "error": null,
  "tool": "stock_price",
  "timestamp": "2025-10-21T15:29:33.361457",
  "metadata": {
    "symbol": "2317"
  }
}


In [7]:
# 測試：使用公司名稱查詢
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "台積電"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "symbol": "2330",
    "company_name": "台灣積體電路製造股份有限公司",
    "current_price": 1480.0,
    "change": 0.0,
    "change_percent": 0.0,
    "volume": 34476,
    "high": 1500.0,
    "low": 1480.0,
    "open": 1485.0,
    "previous_close": 1480.0,
    "last_update": "2025-10-21T14:30:00"
  },
  "error": null,
  "tool": "stock_price",
  "timestamp": "2025-10-21T15:30:33.319399",
  "metadata": {
    "symbol": "台積電"
  }
}


In [8]:
# 測試：查詢ETF
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "取得0050股價"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": false,
  "data": null,
  "error": "無效的股票代號格式: 取得0050股價",
  "tool": "stock_price",
  "timestamp": "2025-10-21T15:30:47.791850",
  "metadata": {
    "symbol": "取得0050股價"
  }
}


### 1.2 模擬股票買賣

In [9]:
# 測試：市價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "0050",
            "quantity": 5000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "buy",
      "symbol": "0050",
      "name": "元大台灣卓越50證券投資信託基金",
      "quantity": 5000,
      "price": 63.2,
      "total_amount": 316000.0,
      "current_price": 63.2,
      "order_type": "market",
      "executed": true,
      "message": "市價單立即成交",
      "timestamp": "2025-10-21T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-21T15:30:55.419355",
  "metadata": {
    "status": "success"
  }
}


In [10]:
# 測試：限價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 2000,
            "price": 500
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "buy",
      "symbol": "2330",
      "name": "台灣積體電路製造股份有限公司",
      "quantity": 2000,
      "price": 500.0,
      "total_amount": 1000000.0,
      "current_price": 1480.0,
      "order_type": "limit",
      "executed": false,
      "message": "限價買單無法成交（出價 500.0 < 市價 1480.0）",
      "timestamp": "2025-10-21T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-21T15:31:04.231020",
  "metadata": {
    "status": "failed"
  }
}


In [ ]:
# 測試：市價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

In [ ]:
# 測試：限價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000,
            "price":550
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

### 1.3 日交易統計

In [ ]:
# 測試：取得日交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_daily_trading",
        {
            "symbol": "2330",
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

## 2. 財務分析工具測試

### 2.1 損益表與資產負債表

In [12]:
# 測試：取得綜合損益表 (一般業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "3231"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "raw_data": {
      "出表日期": "1141021",
      "年度": "114",
      "季別": "2",
      "公司代號": "3231",
      "公司名稱": "緯創",
      "營業收入": "897776520.00",
      "營業成本": "846267473.00",
      "原始認列生物資產及農產品之利益（損失）": "",
      "生物資產當期公允價值減出售成本之變動利益（損失）": "",
      "營業毛利（毛損）": "51509047.00",
      "未實現銷貨（損）益": "",
      "已實現銷貨（損）益": "",
      "營業毛利（毛損）淨額": "51509047.00",
      "營業費用": "25571760.00",
      "其他收益及費損淨額": "",
      "營業利益（損失）": "25937287.00",
      "營業外收入及支出": "7012854.00",
      "稅前淨利（淨損）": "32950141.00",
      "所得稅費用（利益）": "8043469.00",
      "繼續營業單位本期淨利（淨損）": "24906672.00",
      "停業單位損益": "",
      "合併前非屬共同控制股權損益": "",
      "本期淨利（淨損）": "24906672.00",
      "其他綜合損益（淨額）": "-14735117.00",
      "合併前非屬共同控制股權綜合損益淨額": "",
      "本期綜合損益總額": "10171555.00",
      "淨利（淨損）歸屬於母公司業主": "11835345.00",
      "淨利（淨損）歸屬於共同控制下前手權益": "",
      "淨利（淨損）歸屬於非控制權益": "13071327.00",
      "綜合損益總額歸屬於母公司業主": "347386.00",
      "綜合損益總額歸屬於共同控制下前手權益": "",
      "綜合損益總額歸屬於非控制權

In [ ]:
# 測試：取得綜合損益表 (金融業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "2884"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得資產負債表
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_balance_sheet",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.2 公司基本資訊

In [ ]:
# 測試：取得公司基本資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.3 股利資訊

In [ ]:
# 測試：取得股利分配資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_dividend",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得除權息行事曆 (單一公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得除權息行事曆 (所有公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": ""}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 2.4 營收與估值

In [ ]:
# 測試：取得月營收
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_monthly_revenue", {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得估值比率
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_valuation_ratios", {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 3. 交易統計工具測試

### 3.1 月交易統計

In [ ]:
# 測試：取得月交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_trading",
        {"symbol": "0050"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：取得月平均價格
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_average",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 3.2 年交易統計

In [ ]:
# 測試：取得年交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_yearly_trading",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 4. 市場資訊工具測試

### 4.1 融資融券

In [ ]:
# 測試：取得融資融券資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_margin_trading_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.2 即時交易統計

In [ ]:
# 測試：取得即時交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_real_time_trading_stats",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.3 ETF排名

In [ ]:
# 測試：取得ETF定期定額排名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_etf_regular_investment_ranking",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.4 市場指數

In [ ]:
# 測試：取得發行量加權股價指數
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_index_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 4.5 歷史指數

In [ ]:
# 測試：取得歷史指數資料
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_historical_index",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 5. 外資工具測試

### 5.1 外資持股（按產業）

In [ ]:
# 測試：取得外資各產業持股
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_foreign_investment_by_industry",
        {"count": 8}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 5.2 外資持股前20名

In [ ]:
# 測試：取得外資持股前20名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_top_foreign_holdings",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

## 6. 節假日工具測試

### 6.1 節假日資訊

In [ ]:
# 測試：查詢元旦
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：查詢中秋節
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-06"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：查詢一般工作日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

### 6.2 交易日檢查

In [ ]:
# 測試：檢查元旦是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：檢查週六是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-11"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

In [ ]:
# 測試：檢查一般工作日是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))